In [95]:
import pymysql.cursors
import pandas as pd
from sklearn import preprocessing
import numpy as np
import re
import datetime
import scipy.stats as st

# 训练

## 获取全量的比赛平均赔率信息

In [2]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ai', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = "select * from `breadt_match_odd_info` where odd_type = 'avg';"
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_odd_info_df = pd.DataFrame(rows)
    
    connection.close()

In [3]:
train_odd_info_df = train_odd_info_df.drop(['id', 'odd_type'], axis=1)

In [4]:
train_odd_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63438 entries, 0 to 63437
Data columns (total 14 columns):
init_draw_odd     63438 non-null float64
init_lose_odd     63438 non-null float64
init_win_odd      63438 non-null float64
matchid           63438 non-null int64
new_draw_kelly    63438 non-null float64
new_draw_odd      63438 non-null float64
new_draw_rate     63438 non-null float64
new_lose_kelly    63438 non-null float64
new_lose_odd      63438 non-null float64
new_lose_rate     63438 non-null float64
new_win_kelly     63438 non-null float64
new_win_odd       63438 non-null float64
new_win_rate      63438 non-null float64
pay_rate          63438 non-null float64
dtypes: float64(13), int64(1)
memory usage: 6.8 MB


## 获取全量的比赛结果信息

In [5]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ai', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_match_result_list`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_game_list_df = pd.DataFrame(rows)
    
    connection.close()

In [6]:
train_game_list_df['season_year'] = train_game_list_df.apply(lambda row: int(row.season.split('-')[0]), axis=1)
train_game_list_df = train_game_list_df.drop(['id', 'gd', 'gs', 'gn'], axis=1)

In [7]:
# 对team做encode 这个encoder后面预测的时候还会用到
teams = list(set(train_game_list_df['home_team'].values) | set(train_game_list_df['visit_team'].values))
team_encoder = preprocessing.LabelEncoder()
team_encoder.fit(teams)

def encode_team(df):
    df['home_team_encoder'] = team_encoder.transform(df['home_team'])
    df['visit_team_encoder'] = team_encoder.transform(df['visit_team'])
    return df

In [8]:
train_game_list_df['year'] = train_game_list_df.apply(lambda row: row.time.year, axis=1)
train_game_list_df['month'] = train_game_list_df.apply(lambda row: row.time.month, axis=1)
train_game_list_df['day'] = train_game_list_df.apply(lambda row: row.time.day, axis=1)

train_game_list_df = encode_team(train_game_list_df)

In [9]:
train_game_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70570 entries, 0 to 70569
Data columns (total 14 columns):
home_team             70570 non-null object
league                70570 non-null object
matchid               70570 non-null int64
result                70570 non-null int64
season                70570 non-null object
time                  70570 non-null object
turn                  70570 non-null int64
visit_team            70570 non-null object
season_year           70570 non-null int64
year                  70570 non-null int64
month                 70570 non-null int64
day                   70570 non-null int64
home_team_encoder     70570 non-null int64
visit_team_encoder    70570 non-null int64
dtypes: int64(9), object(5)
memory usage: 7.5+ MB


## 获取比赛进球数信息

In [10]:
connection = pymysql.connect(host='localhost', user='root', password='breadt@2019', db='breadt-football-ai', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
with connection.cursor() as cursor:
    sql = 'select * from `breadt_match_goal_info`;'
    cursor.execute(sql)
    rows = cursor.fetchall()
    
    train_match_goal_df = pd.DataFrame(rows)
    
    connection.close()

## 计算2017-2018赛季进球的泊松分布

In [76]:
goals_df = train_match_goal_df[(train_match_goal_df['season'] == '2017-2018') & (train_match_goal_df['league'].astype(int) == 36)]

In [77]:
def get_df(df, gtype, prefix=None):
    t = df.groupby(['team_name']).agg({'goal_num': 'sum', 'league': 'count'})
    
    if prefix is not None:
        return pd.DataFrame({'team_name': t.index.values, gtype: t['goal_num'].values, prefix+'_count': t['league'].values})
    else:
        return pd.DataFrame({'team_name': t.index.values, gtype: t['goal_num'].values})

In [78]:
gd_df = get_df(goals_df[goals_df['goal_type'] == 'gd'], 'gd', 'f')
gs_df = get_df(goals_df[goals_df['goal_type'] == 'gs'], 'gs')
h_gd_df = get_df(goals_df[(goals_df['goal_type'] == 'gd') & (goals_df['home_or_visit'] == 'home')], 'h_gd', 'h')
h_gs_df = get_df(goals_df[(goals_df['goal_type'] == 'gs') & (goals_df['home_or_visit'] == 'home')], 'h_gs')
v_gd_df = get_df(goals_df[(goals_df['goal_type'] == 'gd') & (goals_df['home_or_visit'] == 'visit')], 'v_gd', 'v')
v_gs_df = get_df(goals_df[(goals_df['goal_type'] == 'gs') & (goals_df['home_or_visit'] == 'visit')], 'v_gs')

In [79]:
result_df = pd.merge(gd_df, gs_df, on='team_name', how='left')
result_df = pd.merge(result_df, h_gd_df, on='team_name', how='left')
result_df = pd.merge(result_df, h_gs_df, on='team_name', how='left')
result_df = pd.merge(result_df, v_gd_df, on='team_name', how='left')
result_df = pd.merge(result_df, v_gs_df, on='team_name', how='left')
result_df

,team_name,gd,f_count,gs,h_gd,h_count,h_gs,v_gd,v_count,v_gs
0,伯恩利,39,38,36,17,19,16,22,19,20
1,伯恩茅斯,61,38,45,30,19,26,31,19,19
2,切尔西,38,38,62,16,19,30,22,19,32
3,利物浦,38,38,84,10,19,45,28,19,39
4,南安普敦,56,38,37,26,19,20,30,19,17
5,哈德斯,58,38,28,25,19,16,33,19,12
6,埃弗顿,58,38,44,22,19,28,36,19,16
7,布赖顿,54,38,34,25,19,24,29,19,10
8,斯托克城,68,38,35,30,19,20,38,19,15
9,斯旺西,56,38,28,24,19,17,32,19,11


In [80]:
result_df['gd_rate'] = result_df['gd'] / result_df['f_count']
result_df['gs_rate'] = result_df['gs'] / result_df['f_count']

result_df['h_gd_rate'] = result_df['v_gd'] / result_df['h_count']
result_df['h_gs_rate'] = result_df['v_gs'] / result_df['h_count']

result_df['v_gd_rate'] = result_df['v_gd'] / result_df['v_count']
result_df['v_gs_rate'] = result_df['v_gs'] / result_df['v_count']

result_df

,team_name,gd,f_count,gs,h_gd,h_count,h_gs,v_gd,v_count,v_gs,gd_rate,gs_rate,h_gd_rate,h_gs_rate,v_gd_rate,v_gs_rate
0,伯恩利,39,38,36,17,19,16,22,19,20,1.026316,0.947368,1.157895,1.052632,1.157895,1.052632
1,伯恩茅斯,61,38,45,30,19,26,31,19,19,1.605263,1.184211,1.631579,1.000000,1.631579,1.000000
2,切尔西,38,38,62,16,19,30,22,19,32,1.000000,1.631579,1.157895,1.684211,1.157895,1.684211
3,利物浦,38,38,84,10,19,45,28,19,39,1.000000,2.210526,1.473684,2.052632,1.473684,2.052632
4,南安普敦,56,38,37,26,19,20,30,19,17,1.473684,0.973684,1.578947,0.894737,1.578947,0.894737
5,哈德斯,58,38,28,25,19,16,33,19,12,1.526316,0.736842,1.736842,0.631579,1.736842,0.631579
6,埃弗顿,58,38,44,22,19,28,36,19,16,1.526316,1.157895,1.894737,0.842105,1.894737,0.842105
7,布赖顿,54,38,34,25,19,24,29,19,10,1.421053,0.894737,1.526316,0.526316,1.526316,0.526316
8,斯托克城,68,38,35,30,19,20,38,19,15,1.789474,0.921053,2.000000,0.789474,2.000000,0.789474
9,斯旺西,56,38,28,24,19,17,32,19,11,1.473684,0.736842,1.684211,0.578947,1.684211,0.578947


In [82]:
result_df['avg_gd_rate'] = result_df['gd_rate'].mean()
result_df['avg_gs_rate'] = result_df['gs_rate'].mean()
result_df['avg_h_gd_rate'] = result_df['h_gd_rate'].mean()
result_df['avg_h_gs_rate'] = result_df['h_gs_rate'].mean()
result_df['avg_v_gd_rate'] = result_df['v_gd_rate'].mean()
result_df['avg_v_gs_rate'] = result_df['v_gs_rate'].mean()
result_df

,team_name,gd,f_count,gs,h_gd,h_count,h_gs,v_gd,v_count,v_gs,...,h_gd_rate,h_gs_rate,v_gd_rate,v_gs_rate,avg_gd_rate,avg_gs_rate,avg_h_gd_rate,avg_h_gs_rate,avg_v_gd_rate,avg_v_gs_rate
0,伯恩利,39,38,36,17,19,16,22,19,20,...,1.157895,1.052632,1.157895,1.052632,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
1,伯恩茅斯,61,38,45,30,19,26,31,19,19,...,1.631579,1.000000,1.631579,1.000000,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
2,切尔西,38,38,62,16,19,30,22,19,32,...,1.157895,1.684211,1.157895,1.684211,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
3,利物浦,38,38,84,10,19,45,28,19,39,...,1.473684,2.052632,1.473684,2.052632,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
4,南安普敦,56,38,37,26,19,20,30,19,17,...,1.578947,0.894737,1.578947,0.894737,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
5,哈德斯,58,38,28,25,19,16,33,19,12,...,1.736842,0.631579,1.736842,0.631579,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
6,埃弗顿,58,38,44,22,19,28,36,19,16,...,1.894737,0.842105,1.894737,0.842105,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
7,布赖顿,54,38,34,25,19,24,29,19,10,...,1.526316,0.526316,1.526316,0.526316,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
8,斯托克城,68,38,35,30,19,20,38,19,15,...,2.000000,0.789474,2.000000,0.789474,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
9,斯旺西,56,38,28,24,19,17,32,19,11,...,1.684211,0.578947,1.684211,0.578947,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368


In [83]:
result_df['h_gd_ad'] = result_df['h_gd_rate'] / result_df['avg_h_gd_rate']
result_df['h_gs_ad'] = result_df['h_gs_rate'] / result_df['avg_h_gs_rate']
result_df['v_gd_ad'] = result_df['v_gd_rate'] / result_df['avg_v_gd_rate']
result_df['v_gs_ad'] = result_df['v_gs_rate'] / result_df['avg_v_gs_rate']
result_df

,team_name,gd,f_count,gs,h_gd,h_count,h_gs,v_gd,v_count,v_gs,...,avg_gd_rate,avg_gs_rate,avg_h_gd_rate,avg_h_gs_rate,avg_v_gd_rate,avg_v_gs_rate,h_gd_ad,h_gs_ad,v_gd_ad,v_gs_ad
0,伯恩利,39,38,36,17,19,16,22,19,20,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,0.756014,0.917431,0.756014,0.917431
1,伯恩茅斯,61,38,45,30,19,26,31,19,19,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,1.065292,0.871560,1.065292,0.871560
2,切尔西,38,38,62,16,19,30,22,19,32,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,0.756014,1.467890,0.756014,1.467890
3,利物浦,38,38,84,10,19,45,28,19,39,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,0.962199,1.788991,0.962199,1.788991
4,南安普敦,56,38,37,26,19,20,30,19,17,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,1.030928,0.779817,1.030928,0.779817
5,哈德斯,58,38,28,25,19,16,33,19,12,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,1.134021,0.550459,1.134021,0.550459
6,埃弗顿,58,38,44,22,19,28,36,19,16,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,1.237113,0.733945,1.237113,0.733945
7,布赖顿,54,38,34,25,19,24,29,19,10,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,0.996564,0.458716,0.996564,0.458716
8,斯托克城,68,38,35,30,19,20,38,19,15,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,1.305842,0.688073,1.305842,0.688073
9,斯旺西,56,38,28,24,19,17,32,19,11,...,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368,1.099656,0.504587,1.099656,0.504587


In [92]:
tmp = result_df[['team_name', 'v_gs_ad', 'v_gd_ad', 'h_gs_ad', 'h_gd_ad', 'avg_gd_rate', 'avg_gs_rate', 'avg_h_gd_rate', 'avg_h_gs_rate', 'avg_v_gd_rate', 'avg_v_gs_rate']]
tmp = tmp[tmp['team_name'].isin(['切尔西', '曼城'])]
tmp

,team_name,v_gs_ad,v_gd_ad,h_gs_ad,h_gd_ad,avg_gd_rate,avg_gs_rate,avg_h_gd_rate,avg_h_gs_rate,avg_v_gd_rate,avg_v_gs_rate
2,切尔西,1.46789,0.756014,1.46789,0.756014,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368
10,曼城,2.06422,0.446735,2.06422,0.446735,1.339474,1.339474,1.531579,1.147368,1.531579,1.147368


In [109]:
s_h = tmp.iloc[0]
s_v = tmp.iloc[1]

h_ad = s_h['h_gd_ad'] * s_v['v_gs_ad'] * s_h['avg_h_gd_rate']
v_ad = s_v['h_gd_ad'] * s_h['v_gs_ad'] * s_h['avg_v_gd_rate']
h_ad, v_ad

st.poisson.pmf(0, h_ad) * st.poisson.pmf(0, v_ad)

0.033557482575556095

## 计算2017-2018赛季胜平负的泊松分布

In [120]:
r_df = train_match_goal_df[(train_match_goal_df['season'] == '2017-2018') & (train_match_goal_df['league'].astype(int) == 36) & (train_match_goal_df['goal_type']=='gs')]

In [121]:
def get_df(df, gtype):
    t = df.groupby(['team_name']).agg({'result': 'count'})
    return pd.DataFrame({'team_name': t.index.values, gtype: t['result'].values})

In [122]:
l_df = get_df(r_df[r_df['result'] == 0], 'lose')
d_df = get_df(r_df[r_df['result'] == 1], 'draw')
w_df = get_df(r_df[r_df['result'] == 2], 'win')

h_l_df = get_df(r_df[(r_df['result'] == 0) & (r_df['home_or_visit'] == 'home')], 'h_lose')
h_d_df = get_df(r_df[(r_df['result'] == 1) & (r_df['home_or_visit'] == 'home')], 'h_draw')
h_w_df = get_df(r_df[(r_df['result'] == 2) & (r_df['home_or_visit'] == 'home')], 'h_win')

v_l_df = get_df(r_df[(r_df['result'] == 0) & (r_df['home_or_visit'] == 'visit')], 'v_lose')
v_d_df = get_df(r_df[(r_df['result'] == 1) & (r_df['home_or_visit'] == 'visit')], 'v_draw')
v_w_df = get_df(r_df[(r_df['result'] == 2) & (r_df['home_or_visit'] == 'visit')], 'v_win')

result_df = pd.merge(l_df, d_df, on='team_name', how='left')
result_df = pd.merge(result_df, w_df, on='team_name', how='left')
result_df = pd.merge(result_df, h_l_df, on='team_name', how='left')
result_df = pd.merge(result_df, h_d_df, on='team_name', how='left')
result_df = pd.merge(result_df, h_w_df, on='team_name', how='left')
result_df = pd.merge(result_df, v_l_df, on='team_name', how='left')
result_df = pd.merge(result_df, v_d_df, on='team_name', how='left')
result_df = pd.merge(result_df, v_w_df, on='team_name', how='left')
result_df = result_df.fillna(0)

In [126]:
result_df['count'] = result_df['lose'] + result_df['draw'] + result_df['win']
result_df['h_count'] = result_df['h_lose'] + result_df['h_draw'] + result_df['h_win']
result_df['v_count'] = result_df['v_lose'] + result_df['v_draw'] + result_df['v_win']

result_df['lose_rate'] = result_df['lose'] / result_df['count']
result_df['draw_rate'] = result_df['draw'] / result_df['count']
result_df['win_rate'] = result_df['win'] / result_df['count']

result_df['h_lose_rate'] = result_df['h_lose'] / result_df['h_count']
result_df['h_draw_rate'] = result_df['h_draw'] / result_df['h_count']
result_df['h_win_rate'] = result_df['h_win'] / result_df['h_count']

result_df['v_lose_rate'] = result_df['v_lose'] / result_df['v_count']
result_df['v_draw_rate'] = result_df['v_draw'] / result_df['v_count']
result_df['v_win_rate'] = result_df['v_win'] / result_df['v_count']

result_df['h_lose_ad'] = result_df['h_lose_rate'] / result_df['lose_rate']
result_df['h_draw_ad'] = result_df['h_draw_rate'] / result_df['draw_rate']
result_df['h_win_ad'] = result_df['h_win_rate'] / result_df['win_rate']

result_df['v_lose_ad'] = result_df['v_lose_rate'] / result_df['lose_rate']
result_df['v_draw_ad'] = result_df['v_draw_rate'] / result_df['draw_rate']
result_df['v_win_ad'] = result_df['v_win_rate'] / result_df['win_rate']

result_df

,team_name,lose,draw,win,h_lose,h_draw,h_win,v_lose,v_draw,v_win,...,h_win_rate,v_lose_rate,v_draw_rate,v_win_rate,h_lose_ad,h_draw_ad,h_win_ad,v_lose_ad,v_draw_ad,v_win_ad
0,伯恩利,14,12,12,7.0,5,7,7,7,5,...,0.368421,0.368421,0.368421,0.263158,1.000000,0.833333,1.166667,1.000000,1.166667,0.833333
1,伯恩茅斯,11,11,16,7.0,5,7,4,6,9,...,0.368421,0.210526,0.315789,0.473684,1.272727,0.909091,0.875000,0.727273,1.090909,1.125000
2,切尔西,14,7,17,4.0,4,11,10,3,6,...,0.578947,0.526316,0.157895,0.315789,0.571429,1.142857,1.294118,1.428571,0.857143,0.705882
3,利物浦,9,12,17,0.0,7,12,9,5,5,...,0.631579,0.473684,0.263158,0.263158,0.000000,1.166667,1.411765,2.000000,0.833333,0.588235
4,南安普敦,11,15,12,8.0,7,4,3,8,8,...,0.210526,0.157895,0.421053,0.421053,1.454545,0.933333,0.666667,0.545455,1.066667,1.333333
5,哈德斯,11,10,17,8.0,5,6,3,5,11,...,0.315789,0.157895,0.263158,0.578947,1.454545,1.000000,0.705882,0.545455,1.000000,1.294118
6,埃弗顿,8,10,20,5.0,4,10,3,6,10,...,0.526316,0.157895,0.315789,0.526316,1.250000,0.800000,1.000000,0.750000,1.200000,1.000000
7,布赖顿,6,13,19,4.0,8,7,2,5,12,...,0.368421,0.105263,0.263158,0.631579,1.333333,1.230769,0.736842,0.666667,0.769231,1.263158
8,斯托克城,11,12,15,9.0,5,5,2,7,10,...,0.263158,0.105263,0.368421,0.526316,1.636364,0.833333,0.666667,0.363636,1.166667,1.333333
9,斯旺西,12,9,17,10.0,3,6,2,6,11,...,0.315789,0.105263,0.315789,0.578947,1.666667,0.666667,0.705882,0.333333,1.333333,1.294118


In [128]:
h = result_df.iloc[0]
v = result_df.iloc[1]

# h['h_win_rate'] * v['h_lose_rate'] * 
v,h

(team_name          伯恩茅斯
 lose                 11
 draw                 11
 win                  16
 h_lose                7
 h_draw                5
 h_win                 7
 v_lose                4
 v_draw                6
 v_win                 9
 count                38
 h_count              19
 v_count              19
 lose_rate      0.289474
 draw_rate      0.289474
 win_rate       0.421053
 h_lose_rate    0.368421
 h_draw_rate    0.263158
 h_win_rate     0.368421
 v_lose_rate    0.210526
 v_draw_rate    0.315789
 v_win_rate     0.473684
 h_lose_ad       1.27273
 h_draw_ad      0.909091
 h_win_ad          0.875
 v_lose_ad      0.727273
 v_draw_ad       1.09091
 v_win_ad          1.125
 Name: 1, dtype: object, team_name           伯恩利
 lose                 14
 draw                 12
 win                  12
 h_lose                7
 h_draw                5
 h_win                 7
 v_lose                7
 v_draw                7
 v_win                 5
 count                38
 